In [2]:
import re
import numpy as np
import pandas as pd
import json
import shutil
import gzip
import os
import wget
import tarfile
import time
from time_converter import Time

- We will start with the users,receipts and the brands last
- We will look at the dataset and create structured data model

# User Data Cleaning

In [3]:
user_df = pd.read_csv('csv_files/users.csv', index_col = 0)

In [4]:
users_df = user_df.copy()

In [5]:
users_df.columns

Index(['_id', 'active', 'createdDate', 'lastLogin', 'role', 'signUpSource',
       'state'],
      dtype='object')

- We have a few Dtype objects in our dataset to observe for. While also checking the int64 and float64

In [6]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           495 non-null    object
 1   active        495 non-null    bool  
 2   createdDate   495 non-null    object
 3   lastLogin     433 non-null    object
 4   role          495 non-null    object
 5   signUpSource  447 non-null    object
 6   state         439 non-null    object
dtypes: bool(1), object(6)
memory usage: 27.6+ KB


- Checking first 5 rows we see a few mor data that is in need of cleaning

In [7]:
users_df.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [8]:
def cleanDateLog(id_data, column=None):
    id_list = list(id_data[f'{column}'].to_dict().values())
    times_list = []
    dates_list = []
    for i in range(len(id_list)):
        try: 
            
            date_parsed = id_list[i].replace('}', "").split(':')[1].replace(" ", "")
            date_converted = Time(int(date_parsed[:10]), 'posix').to('dt')
            format_date = "%m/%d/%Y"
            format_time = "%H:%M:%S"
            formated_date = date_converted.strftime(format_date)
            formated_time = date_converted.strftime(format_time)
            times_list.append(formated_time)
            dates_list.append(formated_date)
        except AttributeError:
            
            date_parsed = str(id_list[i])
            dates_list.append(date_parsed)
            
    return dates_list

In [6]:
def cleanIdCpg(id_data, column=None):
#     print(id_data[f'{column}'])
    # create the dictionary into a list to only get the values
    id_list = list(id_data[f'{column}'].to_dict().values())
    cleanedList = []
    for i in range(len(id_list)):
        parsed_name = id_list[i]
        # remove commas colons and split at quotations
        parsed_names = re.findall("'([^']*)'", id_list[i])
        # these were reoccuring strings that i saw
        cleanedList.append(parsed_names[-1])

    return cleanedList

In [851]:
def cleanUserData():    
    user_df = pd.read_csv('csv_files/users.csv', index_col = 0)
    users_df = user_df.copy()

    users_df['_id'] = cleanIdCpg(id_data=users_df, column="_id")
    users_df['createdDate'] = cleanDateLog(id_data=users_df, column="createdDate")
    users_df["lastLogin"] = cleanDateLog(id_data=users_df, column="lastLogin")

    # convert to pandas datetime 
    users_df['createdDate'] = pd.to_datetime(users_df['createdDate'])
    users_df["lastLogin"] = pd.to_datetime(users_df['lastLogin'])
    users_df = users_df.rename(columns = {'_id':'user_id'}, inplace = False)
    users_df.to_csv('clean_data/users_dataset.csv')
    
    return users_df

## Function to Clean User Json File

In [852]:
users_df = cleanUserData()

# Receipt Data Cleaning

In [9]:
# I observed and found thats needed to converted a quick script to loop through the Dates
def convertDates(df):
    new_df = df.copy()
    col_list = list(new_df.columns)
    dates_col = col_list[3:10]
    dates_col.remove(col_list[8])
#     print(dates_col)
    for dates in dates_col:
        df[f'{dates}'] = cleanDateLog(id_data=df, column=f"{dates}")
        
    return df

# receipts_df = convertDates(receipt_df)

In [11]:
def cleanReceipt():
    
    receipt_df = pd.read_csv('csv_files/receipts.csv', index_col = 0)
    receipts_df = convertDates(receipt_df)
    receipts_df['_id'] = cleanIdCpg(receipts_df, column = '_id')
    receipts_df = receipts_df.drop('rewardsReceiptItemList', axis = 1)
    
    return receipts_df

In [881]:
def receiptDf():
    receipt_df = pd.read_csv('csv_files/receipts.csv', index_col = 0)
    receipts_df = convertDates(receipt_df)
    receipts_df['_id'] = cleanIdCpg(receipts_df, column = '_id')
    receipts_df = receipts_df.rename(columns = {'_id':'receipt_id'}, inplace = False)
    date_col = ['createDate','dateScanned','finishedDate','purchaseDate','purchaseDate']
    for col in date_col:
        receipts_df[f'{col}'] = pd.to_datetime(receipts_df[f'{col}'])

    print(receipt_df)
    receipts_df.to_csv('clean_data/receipts_dataset.csv')
    
    return receipts_df

In [882]:
receipt_df = receiptDf()

                           _id  bonusPointsEarned  \
0     5ff1e1eb0a720f0523000575              500.0   
1     5ff1e1bb0a720f052300056b              150.0   
2     5ff1e1f10a720f052300057a                5.0   
3     5ff1e1ee0a7214ada100056f                5.0   
4     5ff1e1d20a7214ada1000561                5.0   
...                        ...                ...   
1114  603cc0630a720fde100003e6               25.0   
1115  603d0b710a720fde1000042a                NaN   
1116  603cf5290a720fde10000413                NaN   
1117  603ce7100a7217c72c000405               25.0   
1118  603c4fea0a7217c72c000389                NaN   

                                bonusPointsEarnedReason  createDate  \
0     Receipt number 2 completed, bonus point schedu...  01/03/2021   
1     Receipt number 5 completed, bonus point schedu...  01/03/2021   
2                            All-receipts receipt bonus  01/03/2021   
3                            All-receipts receipt bonus  01/03/2021   
4       

In [880]:
receipt_df

,receipt_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03,2021-01-03,2021-01-03,01/03/2021,01/03/2021,500.0,2021-01-03,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03,2021-01-03,2021-01-03,01/03/2021,01/03/2021,150.0,2021-01-02,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03,2021-01-03,NaT,01/03/2021,nan,5.0,2021-01-03,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03,2021-01-03,2021-01-03,01/03/2021,01/03/2021,5.0,2021-01-03,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03,2021-01-03,2021-01-03,01/03/2021,01/03/2021,5.0,2021-01-02,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01,2021-03-01,NaT,03/01/2021,nan,25.0,2020-08-17,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,NaN,NaN,2021-03-01,2021-03-01,NaT,03/01/2021,nan,NaN,NaT,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,NaN,NaN,2021-03-01,2021-03-01,NaT,03/01/2021,nan,NaN,NaT,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01,2021-03-01,NaT,03/01/2021,nan,25.0,2020-08-17,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [290]:
total = len(pd.read_json('json_files/receipts.json', lines = True)['rewardsReceiptItemList'])
reference_data = receiptDf()[['receipt_id', 'rewardsReceiptItemList', 'purchasedItemCount']]

In [22]:
receiptquantityLst = reference_data['purchasedItemCount'].to_list()

## Reward Receipt Items Dataset

In [631]:
df = pd.read_json('json_files/receipts.json', lines = True)
def receiptIds(df):  
    # creating receipt Item to json file
    Items_df = df
    reference_data = receiptDf()[['receipt_id', 'rewardsReceiptItemList', 'purchasedItemCount']]
    # how many items are in this dataset
    
    total  = len(Items_df)
    # Receipt Items nested per receipt_id
    
    sample_lst = []
    receipt_labels = []
    missing_data = []
    missing_labels = []
    
        
    # matched to the amount of data that we have currently
    for i in range(total):
        try:
            # create a variable to iterate through the dataframe
            print(i)
            receiptItem_df = pd.DataFrame(Items_df['rewardsReceiptItemList'][i])

            receiptItem_df['receipt_id'] = reference_data['receipt_id'][i]
    #         somedataframe = somedataframe.append(receiptItem_df[i], ignore_index=True)

            # incase we have keyerrors that could mean its only one 
                # each receipt had several subsets of items
            total_items = len(receiptItem_df)

                # each receipt had several items in so we may have multiple indexes 
            for j in range(total_items):
                if j > 1:
                    receipt_labels.append(reference_data['receipt_id'][i])
                    sample_lst.append(receiptItem_df)
                else:
                    receipt_labels.append(reference_data['receipt_id'][i])
                    sample_lst.append(receiptItem_df)

        # incase we have only one we can count it as missing data 
        # that we can look at later
        except:
            print(i, 'missing data/nan')
            missing_labels.append(reference_data['receipt_id'][i])
            missing_data.append(receiptItem_df)

    # Normalize the json data frames by the subsets and stack(concat) them 
    print(f"Missing Data: {len(missing_data)}")
    print(f"Amount of Data: {len(sample_lst)}")
    
    return sample_lst
#     return sample_concat, sample_df_0, sample_df_1, receipt_labels

In [634]:
def receiptItemsDataset(df):  
    # creating receipt Item to json file
    print('appending all datasets')
    Items_df = df
#     reference_data = receiptDf()[['receipt_id', 'rewardsReceiptItemList', 'purchasedItemCount']]

    # how many items are in this dataset
    total  = len(Items_df)
    # Receipt Items nested per receipt_id
    
    sample_lst = pd.DataFrame()
    missing_data = []
    missing_labels = []
    
    receiptItem_col = ['receipt_id','barcode', 'description', 'finalPrice', 'itemPrice', 'needsFetchReview',
       'partnerItemId', 'preventTargetGapPoints', 'quantityPurchased',
       'userFlaggedBarcode', 'userFlaggedNewItem', 'userFlaggedPrice',
       'userFlaggedQuantity', 'needsFetchReviewReason',
       'pointsNotAwardedReason', 'pointsPayerId', 'rewardsGroup',
       'rewardsProductPartnerId', 'userFlaggedDescription',
       'originalMetaBriteBarcode', 'originalMetaBriteDescription', 'brandCode',
       'competitorRewardsGroup', 'discountedItemPrice',
       'originalReceiptItemText']
    
    print(total)
    # matched to the amount of data that we have currently
    for i in range(total):
        receiptItem_df = Items_df[i]
        
        if sample_lst.empty:
            print('start', i)
            sample_lst = sample_lst.append(receiptItem_df)
            
        else:
            print('continuing: ',i)
            sample_lst = sample_lst.append(receiptItem_df, ignore_index=True)

        # incase we have only one we can count it as missing data 
        # that we can look at later

    print('End: ', i)
    print(f"Missing Data: {len(missing_data)}")
    print(f"Amount of Data: {len(sample_lst)}")
    
    return sample_lst[receiptItem_col]

## Functions to place receipt_id on each item
- Create a receipt item dataset 
- Goal: Try not to lose any data unless it is nan
- this function may take a few moments 45(mins)
     - It should print: 
        - End:  6940
        - Missing Data: 0
        - Amount of Data: 1156991
- It is saved to save time as a csv file

In [638]:
receiptItem_dataset.to_csv('csv_files/receiptitems.csv')

In [635]:
# main_df = receiptIds(df)
# receiptItem_dataset = receiptItemsDataset(main_df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
71 missing
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
93 missing
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
149 missing
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
175 missing
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
212 missing
213
214
215
216
216 missing
217
217 missing
218
219
220
221
222
222 missing
223
223 missing
224
225
226
227
228
229
229 missing
230
230 missing
231
232
233
234
234 missing
235
236
236 missing
237
238


1061 missing
1062
1062 missing
1063
1064
1065
1065 missing
1066
1066 missing
1067
1067 missing
1068
1069
1070
1071
1072
1072 missing
1073
1073 missing
1074
1075
1076
1077
1078
1078 missing
1079
1079 missing
1080
1081
1081 missing
1082
1083
1083 missing
1084
1084 missing
1085
1085 missing
1086
1086 missing
1087
1087 missing
1088
1089
1090
1090 missing
1091
1091 missing
1092
1093
1093 missing
1094
1095
1095 missing
1096
1097
1097 missing
1098
1098 missing
1099
1100
1101
1102
1102 missing
1103
1104
1104 missing
1105
1105 missing
1106
1107
1107 missing
1108
1108 missing
1109
1109 missing
1110
1110 missing
1111
1111 missing
1112
1113
1114
1115
1115 missing
1116
1116 missing
1117
1118
1118 missing
Missing Data: 440
Amount of Data: 6941
appending all datasets
start 0
continuing:  1
continuing:  2
continuing:  3
continuing:  4
continuing:  5
continuing:  6
continuing:  7
continuing:  8
continuing:  9
continuing:  10
continuing:  11
continuing:  12
continuing:  13
continuing:  14
continuing:  1

continuing:  448
continuing:  449
continuing:  450
continuing:  451
continuing:  452
continuing:  453
continuing:  454
continuing:  455
continuing:  456
continuing:  457
continuing:  458
continuing:  459
continuing:  460
continuing:  461
continuing:  462
continuing:  463
continuing:  464
continuing:  465
continuing:  466
continuing:  467
continuing:  468
continuing:  469
continuing:  470
continuing:  471
continuing:  472
continuing:  473
continuing:  474
continuing:  475
continuing:  476
continuing:  477
continuing:  478
continuing:  479
continuing:  480
continuing:  481
continuing:  482
continuing:  483
continuing:  484
continuing:  485
continuing:  486
continuing:  487
continuing:  488
continuing:  489
continuing:  490
continuing:  491
continuing:  492
continuing:  493
continuing:  494
continuing:  495
continuing:  496
continuing:  497
continuing:  498
continuing:  499
continuing:  500
continuing:  501
continuing:  502
continuing:  503
continuing:  504
continuing:  505
continuing:  5

continuing:  930
continuing:  931
continuing:  932
continuing:  933
continuing:  934
continuing:  935
continuing:  936
continuing:  937
continuing:  938
continuing:  939
continuing:  940
continuing:  941
continuing:  942
continuing:  943
continuing:  944
continuing:  945
continuing:  946
continuing:  947
continuing:  948
continuing:  949
continuing:  950
continuing:  951
continuing:  952
continuing:  953
continuing:  954
continuing:  955
continuing:  956
continuing:  957
continuing:  958
continuing:  959
continuing:  960
continuing:  961
continuing:  962
continuing:  963
continuing:  964
continuing:  965
continuing:  966
continuing:  967
continuing:  968
continuing:  969
continuing:  970
continuing:  971
continuing:  972
continuing:  973
continuing:  974
continuing:  975
continuing:  976
continuing:  977
continuing:  978
continuing:  979
continuing:  980
continuing:  981
continuing:  982
continuing:  983
continuing:  984
continuing:  985
continuing:  986
continuing:  987
continuing:  9

continuing:  1390
continuing:  1391
continuing:  1392
continuing:  1393
continuing:  1394
continuing:  1395
continuing:  1396
continuing:  1397
continuing:  1398
continuing:  1399
continuing:  1400
continuing:  1401
continuing:  1402
continuing:  1403
continuing:  1404
continuing:  1405
continuing:  1406
continuing:  1407
continuing:  1408
continuing:  1409
continuing:  1410
continuing:  1411
continuing:  1412
continuing:  1413
continuing:  1414
continuing:  1415
continuing:  1416
continuing:  1417
continuing:  1418
continuing:  1419
continuing:  1420
continuing:  1421
continuing:  1422
continuing:  1423
continuing:  1424
continuing:  1425
continuing:  1426
continuing:  1427
continuing:  1428
continuing:  1429
continuing:  1430
continuing:  1431
continuing:  1432
continuing:  1433
continuing:  1434
continuing:  1435
continuing:  1436
continuing:  1437
continuing:  1438
continuing:  1439
continuing:  1440
continuing:  1441
continuing:  1442
continuing:  1443
continuing:  1444
continuing

...

continuing:  5038
continuing:  5039
continuing:  5040
continuing:  5041
continuing:  5042
continuing:  5043
continuing:  5044
continuing:  5045
continuing:  5046
continuing:  5047
continuing:  5048
continuing:  5049
continuing:  5050
continuing:  5051
continuing:  5052
continuing:  5053
continuing:  5054
continuing:  5055
continuing:  5056
continuing:  5057
continuing:  5058
continuing:  5059
continuing:  5060
continuing:  5061
continuing:  5062
continuing:  5063
continuing:  5064
continuing:  5065
continuing:  5066
continuing:  5067
continuing:  5068
continuing:  5069
continuing:  5070
continuing:  5071
continuing:  5072
continuing:  5073
continuing:  5074
continuing:  5075
continuing:  5076
continuing:  5077
continuing:  5078
continuing:  5079
continuing:  5080
continuing:  5081
continuing:  5082
continuing:  5083
continuing:  5084
continuing:  5085
continuing:  5086
continuing:  5087
continuing:  5088
continuing:  5089
continuing:  5090
continuing:  5091
continuing:  5092
continuing

continuing:  5494
continuing:  5495
continuing:  5496
continuing:  5497
continuing:  5498
continuing:  5499
continuing:  5500
continuing:  5501
continuing:  5502
continuing:  5503
continuing:  5504
continuing:  5505
continuing:  5506
continuing:  5507
continuing:  5508
continuing:  5509
continuing:  5510
continuing:  5511
continuing:  5512
continuing:  5513
continuing:  5514
continuing:  5515
continuing:  5516
continuing:  5517
continuing:  5518
continuing:  5519
continuing:  5520
continuing:  5521
continuing:  5522
continuing:  5523
continuing:  5524
continuing:  5525
continuing:  5526
continuing:  5527
continuing:  5528
continuing:  5529
continuing:  5530
continuing:  5531
continuing:  5532
continuing:  5533
continuing:  5534
continuing:  5535
continuing:  5536
continuing:  5537
continuing:  5538
continuing:  5539
continuing:  5540
continuing:  5541
continuing:  5542
continuing:  5543
continuing:  5544
continuing:  5545
continuing:  5546
continuing:  5547
continuing:  5548
continuing

continuing:  5950
continuing:  5951
continuing:  5952
continuing:  5953
continuing:  5954
continuing:  5955
continuing:  5956
continuing:  5957
continuing:  5958
continuing:  5959
continuing:  5960
continuing:  5961
continuing:  5962
continuing:  5963
continuing:  5964
continuing:  5965
continuing:  5966
continuing:  5967
continuing:  5968
continuing:  5969
continuing:  5970
continuing:  5971
continuing:  5972
continuing:  5973
continuing:  5974
continuing:  5975
continuing:  5976
continuing:  5977
continuing:  5978
continuing:  5979
continuing:  5980
continuing:  5981
continuing:  5982
continuing:  5983
continuing:  5984
continuing:  5985
continuing:  5986
continuing:  5987
continuing:  5988
continuing:  5989
continuing:  5990
continuing:  5991
continuing:  5992
continuing:  5993
continuing:  5994
continuing:  5995
continuing:  5996
continuing:  5997
continuing:  5998
continuing:  5999
continuing:  6000
continuing:  6001
continuing:  6002
continuing:  6003
continuing:  6004
continuing

continuing:  6406
continuing:  6407
continuing:  6408
continuing:  6409
continuing:  6410
continuing:  6411
continuing:  6412
continuing:  6413
continuing:  6414
continuing:  6415
continuing:  6416
continuing:  6417
continuing:  6418
continuing:  6419
continuing:  6420
continuing:  6421
continuing:  6422
continuing:  6423
continuing:  6424
continuing:  6425
continuing:  6426
continuing:  6427
continuing:  6428
continuing:  6429
continuing:  6430
continuing:  6431
continuing:  6432
continuing:  6433
continuing:  6434
continuing:  6435
continuing:  6436
continuing:  6437
continuing:  6438
continuing:  6439
continuing:  6440
continuing:  6441
continuing:  6442
continuing:  6443
continuing:  6444
continuing:  6445
continuing:  6446
continuing:  6447
continuing:  6448
continuing:  6449
continuing:  6450
continuing:  6451
continuing:  6452
continuing:  6453
continuing:  6454
continuing:  6455
continuing:  6456
continuing:  6457
continuing:  6458
continuing:  6459
continuing:  6460
continuing

continuing:  6862
continuing:  6863
continuing:  6864
continuing:  6865
continuing:  6866
continuing:  6867
continuing:  6868
continuing:  6869
continuing:  6870
continuing:  6871
continuing:  6872
continuing:  6873
continuing:  6874
continuing:  6875
continuing:  6876
continuing:  6877
continuing:  6878
continuing:  6879
continuing:  6880
continuing:  6881
continuing:  6882
continuing:  6883
continuing:  6884
continuing:  6885
continuing:  6886
continuing:  6887
continuing:  6888
continuing:  6889
continuing:  6890
continuing:  6891
continuing:  6892
continuing:  6893
continuing:  6894
continuing:  6895
continuing:  6896
continuing:  6897
continuing:  6898
continuing:  6899
continuing:  6900
continuing:  6901
continuing:  6902
continuing:  6903
continuing:  6904
continuing:  6905
continuing:  6906
continuing:  6907
continuing:  6908
continuing:  6909
continuing:  6910
continuing:  6911
continuing:  6912
continuing:  6913
continuing:  6914
continuing:  6915
continuing:  6916
continuing

# More data cleaning 
- we have a few nan values we can replace those with the defaults - 
        - Barcode: 4011 [Str]
        - Description: ITEM NOT FOUND [Str]
        - needsFetchReview: False [Bool]
        - finalPrice: 0 [Int]
        - itemPrice: 0 [Int]
        - quantityPurchased: 0 [Int]

In [801]:
def dataCleaning(df)   
    datacleaning = df.copy()
    datacleaning['barcode'] = datacleaning['barcode'].replace(np.nan, '4011')
    datacleaning['description'] = datacleaning['description'].replace(np.nan, 'ITEM NOT FOUND')
    datacleaning['needsFetchReview'] = datacleaning['needsFetchReview'].replace(np.nan, False)
    datacleaning['finalPrice'] = datacleaning['finalPrice'].replace(np.nan, 0)
    datacleaning['itemPrice'] = datacleaning['itemPrice'].replace(np.nan, 0)
    datacleaning['quantityPurchased'] = datacleaning['quantityPurchased'].replace(np.nan, 0)
    datacleaning = datacleaning[(datacleaning['finalPrice'] != '1') & (test_1['barcode'] != '4011')]
    datacleaning['finalPrice'] = datacleaning['finalPrice'].astype('float64')
    datacleaning['itemPrice'] = datacleaning['itemPrice'].astype('float64')
    datacleaning['quantityPurchased'] = datacleaning['quantityPurchased'].astype('int64')
    
    datacleaning['brandCode'] = datacleaning['brandCode'].replace(np.nan, 'NO BRAND CODE')
    datacleaning['userFlaggedBarcode'] = datacleaning['userFlaggedBarcode'].replace(np.nan,'4011')
    datacleaning['userFlaggedQuantity'] = datacleaning['userFlaggedQuantity'].replace(np.nan, 0)
    datacleaning['userFlaggedDescription'] = datacleaning['userFlaggedDescription'].replace(np.nan, 'ITEM NOT FOUND')
    datacleaning['userFlaggedPrice'] = datacleaning['userFlaggedPrice'].replace(np.nan, 0)
    
    return datacleaning

In [802]:
dataCleaning = datacleaning.copy()

## Checking UserFlagged:
 - Barcode
 - Quantity
 - Description
 - Price
 
Some of the data might be found there but its best to check if there are any nan values to switch to default

In [ ]:
dataCleaning['userFlaggedBarcode'] = dataCleaning['userFlaggedBarcode'].replace(np.nan,'4011')
dataCleaning['userFlaggedQuantity'] = dataCleaning['userFlaggedQuantity'].replace(np.nan, 0)
dataCleaning['userFlaggedDescription'] = dataCleaning['userFlaggedDescription'].replace(np.nan, 'ITEM NOT FOUND')
dataCleaning['userFlaggedPrice'] = dataCleaning['userFlaggedPrice'].replace(np.nan, 0)

In [803]:
dataCleaning['userFlaggedBarcode'] = dataCleaning['userFlaggedBarcode'].replace(np.nan,'4011')
dataCleaning['userFlaggedQuantity'] = dataCleaning['userFlaggedQuantity'].replace(np.nan, 0)
dataCleaning['userFlaggedDescription'] = dataCleaning['userFlaggedDescription'].replace(np.nan, 'ITEM NOT FOUND')
dataCleaning['userFlaggedPrice'] = dataCleaning['userFlaggedPrice'].replace(np.nan, 0)

In [804]:
print('Description Count:',
      len(dataCleaning[(dataCleaning['description'] == 'ITEM NOT FOUND') 
                       & (dataCleaning['userFlaggedDescription'] != 'ITEM NOT FOUND')]))

# Description Count: 66

Description Count: 66


In [805]:
print("Barcode Count: ",
    len(dataCleaning[(dataCleaning['barcode'] == '4011') 
                       & (dataCleaning['userFlaggedBarcode'] != '4011')]))

# Barcode Count:  0

Barcode Count:  0


## We can comfortably remove userflag barcode and description

In [806]:
dataCleaning = dataCleaning.drop(columns=['userFlaggedBarcode', 'userFlaggedDescription'])

In [807]:
receipts_itemdf = dataCleaning.copy()

In [862]:
receipts_itemdf.to_pickle('clean_data/receiptitems_dataset.zip')

# Brand Data Cleaning

In [712]:
def BrandIdCpg(id_data, column=None):
#     print(id_data[f'{column}'])
    # create the dictionary into a list to only get the values
    id_list = list(id_data[f'{column}'].to_dict().values())
    cleanedList = []
    for i in range(len(id_list)):
        parsed_name = id_list[i]
        # remove commas colons and split at quotations
        parsed_names = re.findall("'([^']*)'", id_list[i])
        # these were reoccuring strings that i saw
        remove_list = ['$ref', 'Cogs','$id', '$oid']
        # remove the extra strings and keep the actual id #
        res = [i for i in parsed_names if i not in remove_list]
        cleanedList.append(res[0])
        
    return cleanedList

In [713]:
brand_df = pd.read_csv('csv_files/brands.csv', index_col = 0)

In [867]:
brand_df = pd.read_csv('csv_files/brands.csv', index_col = 0)
brands_df = brand_df.copy()
brands_df['_id'] = BrandIdCpg(brands_df, column = '_id')
brands_df['cpg'] = BrandIdCpg(brands_df, column = 'cpg')
brands_df = brands_df.rename(columns = {'_id':'brand_id'}, inplace = False)
brands_df = brands_df.rename(columns = {'brandCode':'brandCode_id'}, inplace = False)
brands_df = brands_df[(brands_df['name'].str.contains("test") == False)]
brands_df['topBrand'] = brands_df['topBrand'].replace(np.nan, 0)
brands_df['categoryCode'] = brands_df['category'].str.upper()
brands_df['barcode'] = brands_df['barcode'].astype('object')
brands_df.to_csv('clean_data/brands_dataset.csv')
len(brands_df)

1167

## Data Info

In [740]:
users_df.info()
# Data Model Users
# user_id = [str] pk
# active = [bool]
# createdate = [datetime]
# lastlogin = [datetime]
# role = [str]
# signupsource = [str]
# state = [str]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   user_id       495 non-null    object        
 1   active        495 non-null    bool          
 2   createdDate   495 non-null    datetime64[ns]
 3   lastLogin     433 non-null    datetime64[ns]
 4   role          495 non-null    object        
 5   signUpSource  447 non-null    object        
 6   state         439 non-null    object        
dtypes: bool(1), datetime64[ns](2), object(4)
memory usage: 27.6+ KB


In [742]:
brands_df.info()
# Data Model
# brand_id = [str] pk
# barcode = [str]
# category = [str]
# cpg = [str]
# name = [str]
# topBrand = [int]
# brandCode = [str]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 738 entries, 1 to 1165
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   brand_id      738 non-null    object 
 1   barcode       738 non-null    object 
 2   category      583 non-null    object 
 3   categoryCode  583 non-null    object 
 4   cpg           738 non-null    object 
 5   name          738 non-null    object 
 6   topBrand      738 non-null    float64
 7   brandCode     540 non-null    object 
dtypes: float64(1), object(7)
memory usage: 51.9+ KB


In [746]:
receipts_df.info()
# Data Model Receipts

# receipt_id = [str] pk
# bonusPointsEarned = [int] 
# bonusPointsEarnedReason = [int] 
# createDate = [datetime]
# dateScanned = [datetime]
# finishedDate = [datetime]
# modifyDate = [datetime]
# pointsAwardedDate = [datetime]
# pointsEarned = [int]
# purchaseDate = [datetime]
# purchasedItemCount = [int]
# rewardsReceiptStatus = [str]
# totalSpent = [float] 
# user_id = [str] fk

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   receipt_id               1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             1119 non-null   object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        1119 non-null   object 
 8   pointsEarned             609 non-null    float64
 9   purchaseDate             1119 non-null   object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1119 non-null   object 
 13  totalSpent               684 non-null    float64
 14  userId                  

In [790]:
receipts_itemdf.info()
# Data Model Receipt Items 
# receipt_id = [str] fk
# brandCode_id = [str] fk

# barcode_id = [str] 
# description = [str]
# finalPrice = [int]
# itemPrice = [int]
# needsFetchReview = [bool]
# partnerItemId = [int]
# preventTargetGapPoints = [int]
# quantityPurchased = [int]
# userFlaggedBarcode = [str]
# userFlaggedNewItem = [int]
# userFlaggedPrice = [float]
# userFlaggedQuantity = [int]
# needsFetchReviewReason = [str]
# pointsNotAwardedReason = []
# pointsPayerId = [str]
# rewardsGroup = [str]
# rewardsProductPartnerId = [str]
# userFlaggedDescription = [str]
# originalMetaBriteBarcode = [str]
# originalMetaBriteDescription = [str]
# brandCode_id = [str] pk
# competitorRewardsGroup = [str]
# discountedItemPrice = [int]
# originalReceiptItemText = [str]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 375215 entries, 2 to 1156990
Data columns (total 23 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   receipt_id                    375215 non-null  object 
 1   barcode                       375215 non-null  object 
 2   description                   375215 non-null  object 
 3   finalPrice                    375215 non-null  float64
 4   itemPrice                     375215 non-null  float64
 5   needsFetchReview              375215 non-null  bool   
 6   partnerItemId                 375215 non-null  object 
 7   preventTargetGapPoints        9793 non-null    object 
 8   quantityPurchased             375215 non-null  int64  
 9   userFlaggedNewItem            343 non-null     object 
 10  userFlaggedPrice              375215 non-null  object 
 11  userFlaggedQuantity           375215 non-null  float64
 12  needsFetchReviewReason        1076 non-null

In [1134]:
len(brands_df)

738

In [1136]:
"""SELECT tablename1.colunmname, tablename2.columnnmae    
FROM tablename1  
JOIN tablename2  
ON tablenmae1.colunmnam = tablename2.columnnmae
ORDER BY columnname; """

186

In [28]:
"""select brandName  from brand as bN,
  (select sum(finalPrice) from table2) as pricesums JOIN ON brand.barcode = receiptItem.barcode
  GROUPBY brandName ASC """

'select brandName  from brand as bN,\n  (select sum(finalPrice) from table2) as pricesums JOIN ON brand.barcode = receiptItem.barcode\n  GROUPBY brandName ASC'